In [32]:
import os

import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
import warnings
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import *
from statsmodels.stats.multicomp import pairwise_tukeyhsd

warnings.filterwarnings("ignore")

output_dir = "pp"
os.makedirs(output_dir, exist_ok=True) 

In [33]:
file_paths = [f"error/hirasaki_{i}_errors.csv" for i in range(10)]
dataframes = [pd.read_csv(file_path, index_col=0) for file_path in file_paths]

In [34]:
frame_wise_avg_mpjpe_combinations_revised = {n: [] for n in range(2, 6)}

for n in range(5):
    for combo_indices in combinations(range(5), n + 2):
        for df in dataframes:
            combo_col_name = f"{''.join(map(str, combo_indices))}_mpjpe"
            if combo_col_name in df.columns:
                frame_wise_avg_mpjpe_combinations_revised[n + 2].extend(df[combo_col_name].values)

print("Number of combinations for each n:")
for n, values in frame_wise_avg_mpjpe_combinations_revised.items():
    print(f"n={n}: {len(values)}")

Number of combinations for each n:
n=2: 18440
n=3: 18440
n=4: 9220
n=5: 1844


In [35]:
raw_combination_data = []
for df in dataframes:
    for col in df.columns:
        if "_mpjpe" in col or "_angle" in col:
            error_type = "MPJPE" if "_mpjpe" in col else col.split('_')[1] + " " + col.split('_')[2]
            if len(col.split('_')) == 6:
                error_type += " " + col.split('_')[3]
            error_type = error_type.replace("NKEE", "KNEE")
            for value in df[col].values:
                raw_combination_data.append({"Combination": col.split('_')[0], "Error Type": error_type, "Error Value": value})

raw_combination_df = pd.DataFrame(raw_combination_data)
raw_combination_df.head()

,Combination,Error Type,Error Value
0,01,MPJPE,31.56
1,01,MPJPE,32.11
2,01,MPJPE,35.39
3,01,MPJPE,33.97
4,01,MPJPE,30.75


In [36]:
plt.figure(figsize=(16, 6))
sns.boxplot(data=raw_combination_df[raw_combination_df["Error Type"] == "MPJPE"], x="Combination", y="Error Value", palette="coolwarm")
plt.xticks(rotation=90)
plt.title("RA-MPJPE for all camera combinations")
plt.xlabel("Camera Combinations")
plt.ylabel("RA-MPJPE (mm)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{output_dir}/05_all_combinations_mpjpe.png")
plt.close()

In [37]:
plt.figure(figsize=(16, 6))
sns.boxplot(data=raw_combination_df[raw_combination_df["Error Type"] != "MPJPE"], x="Combination", y="Error Value", hue="Error Type", palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Angle Error for all camera combinations")
plt.xlabel("Camera Combinations")
plt.ylabel("Angle Error (degrees)")
plt.legend(title="Angle Type")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{output_dir}/05_all_combinations_angle.png")
plt.close()

In [38]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [39]:
mpjpe_df = raw_combination_df[raw_combination_df["Error Type"] == "MPJPE"]
mpjpe_df_no_outliers = remove_outliers(mpjpe_df, "Error Value")

angle_error_df = raw_combination_df[raw_combination_df["Error Type"] != "MPJPE"]
angle_error_df_no_outliers = remove_outliers(angle_error_df, "Error Value")

In [40]:
plt.figure(figsize=(16, 6))
sns.boxplot(data=mpjpe_df_no_outliers, x="Combination", y="Error Value", palette="coolwarm")
plt.xticks(rotation=90)
plt.title("RA-MPJPE for all camera combinations (without outliers)")
plt.xlabel("Camera Combinations")
plt.ylabel("RA-MPJPE (mm)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{output_dir}/05_all_combinations_mpjpe_no_outliers.png")
plt.close()

In [41]:
plt.figure(figsize=(16, 6))
sns.boxplot(data=angle_error_df_no_outliers, x="Combination", y="Error Value", hue="Error Type", palette="coolwarm")
plt.xticks(rotation=90)
plt.title("Angle Error for all camera combinations (without outliers)")
plt.xlabel("Camera Combinations")
plt.ylabel("Angle Error (degrees)")
plt.legend(title="Angle Type")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{output_dir}/05_all_combinations_angle_no_outliers.png")
plt.close()

In [42]:
combination_order = raw_combination_df["Combination"].unique()

mean_values = raw_combination_df[raw_combination_df["Error Type"] == "MPJPE"].groupby("Combination")["Error Value"].mean().reindex(combination_order)
std_values = raw_combination_df[raw_combination_df["Error Type"] == "MPJPE"].groupby("Combination")["Error Value"].std().reindex(combination_order)

plt.figure(figsize=(16, 6))
plt.bar(mean_values.index, mean_values, yerr=std_values, capsize=5, alpha=0.7)
plt.xticks(rotation=90)
plt.title("Mean and Standard Deviation of RA-MPJPE for all camera combinations")
plt.xlabel("Camera Combination")
plt.ylabel("RA-MPJPE (mm)")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(f"{output_dir}/06_all_combinations_mpjpe_mean_std.png")
plt.close()

In [43]:
x = range(len(mean_values))
bar_width = 0.4

plt.figure(figsize=(16, 6))
mean_bars = plt.bar(x, mean_values, width=bar_width, label='Mean (mm)', alpha=0.7)
std_bars = plt.bar([p + bar_width for p in x], std_values, width=bar_width, label='Std (mm)', alpha=0.7)

for bar, mean in zip(mean_bars, mean_values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{mean:.2f}', ha='center', va='bottom', fontsize=10, color='black')
for bar, std in zip(std_bars, std_values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{std:.2f}', ha='center', va='bottom', fontsize=10, color='black')

plt.xticks([p + bar_width / 2 for p in x], mean_values.index, rotation=90)
plt.title("Mean and Standard Deviation of RA-MPJPE for all camera combinations")
plt.xlabel("Camera Combination")
plt.ylabel("RA-MPJPE (mm)")
plt.legend(title="Metric")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(f"{output_dir}/06_all_combinations_mpjpe_mean_std_separate.png")
plt.close()

In [44]:
camera04_two_camera_data = raw_combination_df[
    (raw_combination_df["Error Type"] == "MPJPE") &
    (raw_combination_df["Combination"].str.len() == 2) &
    (raw_combination_df["Combination"].str.contains("04"))
]["Error Value"]

other_two_camera_data = raw_combination_df[
    (raw_combination_df["Error Type"] == "MPJPE") &
    (raw_combination_df["Combination"].str.len() == 2) &
    (~raw_combination_df["Combination"].str.contains("04"))
]["Error Value"]

ks_stat_two_camera, ks_pvalue_two_camera = ks_2samp(camera04_two_camera_data, other_two_camera_data)
print(f"Two Camera Combination (04 vs Others):")
print(f"KS Statistic: {ks_stat_two_camera:.4f}, p-value: {ks_pvalue_two_camera:.4f}")

Two Camera Combination (04 vs Others):
KS Statistic: 0.5493, p-value: 0.0000


In [45]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(camera04_two_camera_data, bins=50, alpha=0.7, label="Camera 04 Combinations", density=True)
plt.hist(other_two_camera_data, bins=50, alpha=0.7, label="Other 2-Camera Combinations", density=True)
plt.title("Histogram of RA-MPJPE Distributions (2-Camera Combinations)")
plt.xlabel("RA-MPJPE (mm)")
plt.ylabel("Density")
plt.legend()

plt.subplot(1, 2, 2)
camera04_sorted = sorted(camera04_two_camera_data)
other_sorted = sorted(other_two_camera_data)
plt.plot(camera04_sorted, np.linspace(0, 1, len(camera04_sorted)), label="Camera 04 Combinations")
plt.plot(other_sorted, np.linspace(0, 1, len(other_sorted)), label="Other 2-Camera Combinations")
plt.title("Cumulative Distribution Function (CDF) of RA-MPJPE (2-Camera Combinations)")
plt.xlabel("RA-MPJPE (mm)")
plt.ylabel("Cumulative Probability")
plt.legend()

plt.tight_layout()
plt.savefig(f"{output_dir}/07_two_camera_combinations.png")
plt.close()

In [46]:
stat_df = pd.DataFrame()
stat_df['Camera 04'] = camera04_two_camera_data.describe()
stat_df['Others'] = other_two_camera_data.describe()
display(stat_df)

,Camera 04,Others
count,1844.00,16596.00
mean,65.91,33.85
std,51.71,9.95
min,26.66,19.04
25%,37.82,27.85
50%,46.76,31.52
75%,75.99,36.55
max,458.91,157.03


In [47]:
mpjpe_df = raw_combination_df[(raw_combination_df["Error Type"] == "MPJPE") & (raw_combination_df["Combination"] != "04")]
display(mpjpe_df)

,Combination,Error Type,Error Value
0,01,MPJPE,31.56
1,01,MPJPE,32.11
2,01,MPJPE,35.39
3,01,MPJPE,33.97
4,01,MPJPE,30.75
...,...,...,...
334439,01234,MPJPE,34.52
334440,01234,MPJPE,36.56
334441,01234,MPJPE,40.43
334442,01234,MPJPE,41.69


In [48]:
mean_values = [mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"].mean() for n in range(2, 6)]
std_values = [mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"].std() for n in range(2, 6)]

print("Mean values for each n:")
for n, mean, std in zip(range(2, 6), mean_values, std_values):
    print(f"n={n}: {mean:.2f} ± {std:.2f}")

reduction_rates = [(mean_values[n] - mean_values[n + 1]) / mean_values[n] * 100 for n in range(3)]
print(f"Decreasing rates for each n:")
for n, rate in enumerate(reduction_rates):
    print(f"{n + 2} -> {n + 3}: {rate:.2f}")

diminishing_rates = [reduction_rates[n] - reduction_rates[n + 1] for n in range(2)]
print(f"Diminishing rates for each n:")
for n, rate in enumerate(diminishing_rates):
    print(f"{n + 2} -> {n + 3} -> {n + 4}: {rate:.2f}")

Mean values for each n:
n=2: 33.85 ± 9.95
n=3: 29.58 ± 7.26
n=4: 27.35 ± 6.34
n=5: 26.18 ± 6.18
Decreasing rates for each n:
2 -> 3: 12.61
3 -> 4: 7.53
4 -> 5: 4.30
Diminishing rates for each n:
2 -> 3 -> 4: 5.09
3 -> 4 -> 5: 3.22


In [49]:
plt.figure(figsize=(8, 6))
plt.errorbar(range(2, 6), mean_values, yerr=std_values, fmt='o-', capsize=5, label="Mean MPJPE")
for i, (x, y, std) in enumerate(zip(range(2, 6), mean_values, std_values)):
    plt.text(x, y + std + 0.5, f"{y:.2f}", ha='center', fontsize=10)
plt.title("Average RA-MPJPE with Standard Deviation", fontsize=14)
plt.xlabel("Number of Viewpoints (n)", fontsize=12)
plt.ylabel("Average RA-MPJPE (mm)", fontsize=12)
plt.xticks(range(2, 6), labels=[f"n={n}" for n in range(2, 6)], fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10)
plt.grid(alpha=0.5)
plt.tight_layout()
plt.savefig(f"{output_dir}/08_mpjpe_mean_std.png")
plt.close()

In [50]:
plt.figure()
plt.bar(range(2, 5), reduction_rates, tick_label=["2->3", "3->4", "4->5"])
plt.title("Reduction Rates of RA-MPJPE", fontsize=14)
plt.xlabel("Transitions between n")
plt.ylabel("Reduction Rate (%)")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(f"{output_dir}/08_reduction_rates.png")
plt.close()

In [51]:
plt.figure(figsize=(8, 5))
boxplot_data = []
q3_values = []

for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    boxplot_data.append(values)
    q3_values.append(values.quantile(0.75))

plt.boxplot(boxplot_data, labels=[f"n={n}" for n in range(2, 6)], showfliers=False)

plt.plot(range(1, 5), mean_values, marker='o', linestyle='-', color='blue', label="Mean MPJPE")

for i, (start, end) in enumerate(zip(mean_values[:-1], mean_values[1:])):
    diff = start - end
    rate = diff / start * 100
    plt.text(i + 1.5, (start + end) / 2 + 1, f"-{diff:.2f}\n({rate:.1f}%)", ha='center', fontsize=10, color='red')

for i, (mean, q3) in enumerate(zip(mean_values, q3_values)):
    plt.text(i + 0.88, q3 + 0.6, f"{mean:.2f}", ha='center', fontsize=10, color='blue')

plt.title("RA-MPJPE Distribution with Mean Values and Reductions", fontsize=14)
plt.xlabel("Number of Viewpoints (n)", fontsize=12)
plt.ylabel("RA-MPJPE (mm)", fontsize=12)
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.5)
plt.tight_layout()
plt.savefig(f"{output_dir}/08_mpjpe_distribution.png")
plt.close()


In [52]:
for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    plt.figure(figsize=(6, 6))
    probplot(values, dist="norm", plot=plt)
    plt.title(f"Q-Q Plot for n={n}")
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel("Ordered Values")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/09_qqplot_n{n}.png")
    plt.close()

In [53]:
for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    stat, p = shapiro(values)
    print(f"n={n}: stat={stat:.4f}, p={p:.4f}")

for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    plt.figure(figsize=(6, 4))
    sns.histplot(values, kde=True, label=f"n={n}")
    mean_value = values.mean()
    plt.axvline(mean_value, color="red", linestyle="--", label=f"Mean = {mean_value:.2f}")
    plt.title(f"RA-MPJPE Distribution for n={n}")
    plt.xlabel("RA-MPJPE (mm)")
    plt.ylabel("Frequency")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/10_hist_n{n}.png")
    plt.close()

n=2: stat=0.7581, p=0.0000
n=3: stat=0.6320, p=0.0000
n=4: stat=0.5394, p=0.0000
n=5: stat=0.4898, p=0.0000


In [54]:
trimmed_results = []
for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    trimmed_values = values[values <= values.quantile(0.99)]
    stat, p = shapiro(trimmed_values)
    trimmed_results.append((n, stat, p))

for n, stat, p in trimmed_results:
    print(f"n={n} (trimmed): stat={stat:.4f}, p={p:.4f}")

for n in range(2, 6):
    values = mpjpe_df[mpjpe_df["Combination"].str.len() == n]["Error Value"]
    trimmed_values = values[values <= values.quantile(0.99)]
    plt.figure(figsize=(6, 4))
    sns.histplot(trimmed_values, kde=True, label=f"n={n} (trimmed)", color="green", bins=20)
    mean_value = trimmed_values.mean()
    plt.axvline(mean_value, color="red", linestyle="--", label=f"Mean = {mean_value:.2f}")
    plt.title(f"Trimmed RA-MPJPE Distribution for n={n} (99th Percentile Removed)", fontsize=14)
    plt.xlabel("RA-MPJPE (mm)", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(alpha=0.5)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/11_hist_trimmed_n{n}.png")
    plt.close()


n=2 (trimmed): stat=0.8846, p=0.0000
n=3 (trimmed): stat=0.9337, p=0.0000
n=4 (trimmed): stat=0.9852, p=0.0000
n=5 (trimmed): stat=0.9537, p=0.0000


In [55]:
right_elbow_df = raw_combination_df[(raw_combination_df["Error Type"] == "RIGHT ELBOW") & (raw_combination_df["Combination"] != "04")]
left_elbow_df = raw_combination_df[(raw_combination_df["Error Type"] == "LEFT ELBOW") & (raw_combination_df["Combination"] != "04")]
right_knee_df = raw_combination_df[(raw_combination_df["Error Type"] == "RIGHT KNEE") & (raw_combination_df["Combination"] != "04")]
left_knee_df = raw_combination_df[(raw_combination_df["Error Type"] == "LEFT KNEE") & (raw_combination_df["Combination"] != "04")]
right_knee_up_df = raw_combination_df[(raw_combination_df["Error Type"] == "RIGHT KNEE UP") & (raw_combination_df["Combination"] != "04")]
left_knee_up_df = raw_combination_df[(raw_combination_df["Error Type"] == "LEFT KNEE UP") & (raw_combination_df["Combination"] != "04")]

angle_dict = {
    "Right Elbow": right_elbow_df,
    "Left Elbow": left_elbow_df,
    "Right Knee": right_knee_df,
    "Left Knee": left_knee_df,
    "Right Knee Up": right_knee_up_df,
    "Left Knee Up": left_knee_up_df,
}

for angle, df in angle_dict.items():
    print(f"--- {angle} Error Analysis: ---")
    mean_values = [df[df["Combination"].str.len() == n]["Error Value"].mean() for n in range(2, 6)]
    std_values = [df[df["Combination"].str.len() == n]["Error Value"].std() for n in range(2, 6)]

    print("Mean values for each n:")
    for n, mean, std in zip(range(2, 6), mean_values, std_values):
        print(f"  n={n}: {mean:.2f} ± {std:.2f}")

    reduction_rates = [(mean_values[n] - mean_values[n + 1]) / mean_values[n] * 100 for n in range(3)]
    print(f"Decreasing rates for each n:")
    for n, rate in enumerate(reduction_rates):
        print(f"  {n + 2} -> {n + 3}: {rate:.2f}")

    diminishing_rates = [reduction_rates[n] - reduction_rates[n + 1] for n in range(2)]
    print(f"Diminishing rates for each n:")
    for n, rate in enumerate(diminishing_rates):
        print(f"  {n + 2} -> {n + 3} -> {n + 4}: {rate:.2f}")
    
    print()

for angle, df in angle_dict.items():
    plt.figure(figsize=(8, 5))
    boxplot_data = []
    q3_values = []

    for n in range(2, 6):
        values = df[df["Combination"].str.len() == n]["Error Value"]
        boxplot_data.append(values)
        q3_values.append(values.quantile(0.75))

    plt.boxplot(boxplot_data, labels=[f"n={n}" for n in range(2, 6)], showfliers=False)

    plt.plot(range(1, 5), mean_values, marker='o', linestyle='-', color='blue', label="Mean MPJPE")

    for i, (start, end) in enumerate(zip(mean_values[:-1], mean_values[1:])):
        diff = start - end
        rate = diff / start * 100
        plt.text(i + 1.5, (start + end) / 2 + 0.5, f"-{diff:.2f}\n({rate:.1f}%)", ha='center', fontsize=10, color='red')

    for i, (mean, q3) in enumerate(zip(mean_values, q3_values)):
        plt.text(i + 0.88, q3 + 0.2, f"{mean:.2f}", ha='center', fontsize=10, color='blue')

    plt.title(f"{angle} Angle Error Distribution with Mean Values and Reductions", fontsize=14)
    plt.xlabel("Number of Viewpoints (n)", fontsize=12)
    plt.ylabel("Angle Error", fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(axis='y', alpha=0.5)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/12_{angle.lower().replace(' ', '_')}_distribution.png")
    plt.close()
    
combination_order = raw_combination_df["Combination"].unique()

for angle in ["RIGHT ELBOW", "LEFT ELBOW", "RIGHT KNEE", "LEFT KNEE", "RIGHT KNEE UP", "LEFT KNEE UP"]:
    mean_values = raw_combination_df[raw_combination_df["Error Type"] == angle].groupby("Combination")["Error Value"].mean().reindex(combination_order)
    std_values = raw_combination_df[raw_combination_df["Error Type"] == angle].groupby("Combination")["Error Value"].std().reindex(combination_order)

    plt.figure(figsize=(16, 6))
    plt.bar(mean_values.index, mean_values, yerr=std_values, capsize=5, alpha=0.7)
    plt.xticks(rotation=90)
    plt.title("Mean and Standard Deviation of Angle Errors by Camera Combination for " + angle, fontsize=14)
    plt.xlabel("Camera Combination")
    plt.ylabel("Angle Error (degrees)")
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/13_{angle.lower().replace(' ', '_')}_mean_std.png")
    plt.close()
    
    x = range(len(mean_values))
    bar_width = 0.4

    plt.figure(figsize=(16, 6))
    mean_bars = plt.bar(x, mean_values, width=bar_width, label='Mean (mm)', alpha=0.7)
    std_bars = plt.bar([p + bar_width for p in x], std_values, width=bar_width, label='Std (mm)', alpha=0.7)

    for bar, mean in zip(mean_bars, mean_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{mean:.2f}', ha='center', va='bottom', fontsize=7, color='black')
    for bar, std in zip(std_bars, std_values):
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{std:.2f}', ha='center', va='bottom', fontsize=7, color='black')

    plt.xticks([p + bar_width / 2 for p in x], mean_values.index, rotation=90)
    plt.title("Mean and Standard Deviation of Angle Errors by Camera Combination for " + angle, fontsize=14)
    plt.xlabel("Camera Combination")
    plt.ylabel("Angle Error (degrees)")
    plt.legend(title="Metric")
    plt.grid(axis='y')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/13_{angle.lower().replace(' ', '_')}_mean_std_separate.png")
    plt.close()

--- Right Elbow Error Analysis: ---
Mean values for each n:
  n=2: 6.00 ± 7.11
  n=3: 4.37 ± 4.22
  n=4: 3.59 ± 2.96
  n=5: 3.11 ± 2.14
Decreasing rates for each n:
  2 -> 3: 27.17
  3 -> 4: 17.82
  4 -> 5: 13.28
Diminishing rates for each n:
  2 -> 3 -> 4: 9.36
  3 -> 4 -> 5: 4.54

--- Left Elbow Error Analysis: ---
Mean values for each n:
  n=2: 5.50 ± 9.74
  n=3: 4.04 ± 7.55
  n=4: 3.27 ± 6.75
  n=5: 2.78 ± 6.52
Decreasing rates for each n:
  2 -> 3: 26.43
  3 -> 4: 19.03
  4 -> 5: 15.24
Diminishing rates for each n:
  2 -> 3 -> 4: 7.40
  3 -> 4 -> 5: 3.79

--- Right Knee Error Analysis: ---
Mean values for each n:
  n=2: 3.57 ± 2.72
  n=3: 3.18 ± 2.15
  n=4: 2.99 ± 1.95
  n=5: 2.89 ± 1.83
Decreasing rates for each n:
  2 -> 3: 10.93
  3 -> 4: 5.87
  4 -> 5: 3.28
Diminishing rates for each n:
  2 -> 3 -> 4: 5.06
  3 -> 4 -> 5: 2.59

--- Left Knee Error Analysis: ---
Mean values for each n:
  n=2: 3.09 ± 3.09
  n=3: 2.48 ± 2.28
  n=4: 2.27 ± 2.08
  n=5: 2.15 ± 2.01
Decreasing rates f